In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
WINDOW_SIZE = 25
NUMBER_OF_FEATURES = 6
NUMBER_OF_CLASSES = 2

# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TEST2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'


# v0 data (not normalized, use to generate more data)
ALL_X_TEST_CURRVEL_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CURRVEL-V0.npy'
ALL_Y_TEST_CURRVEL_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CURRVEL-V0.npy'
ALL_X_TEST_ALLVEL_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-ALLVEL-V0.npy'
ALL_Y_TEST_ALLVEL_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-ALLVEL-V0.npy'

# v1 data (0-1 normalized data)
ALL_X_TEST_CURRVEL_NORM1_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CURRVEL-V1.npy'
ALL_Y_TEST_CURRVEL_NORM1_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CURRVEL-V1.npy'
ALL_X_TEST_ALLVEL_NORM1_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-ALLVEL-V1.npy'
ALL_Y_TEST_ALLVEL_NORM1_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-ALLVEL-V1.npy'

# v2 data (only 4 features, normalized data)
ALL_X_TEST_NOVEL_NORM_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-NOVEL-V2.npy'
ALL_Y_TEST_NOVEL_NORM_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-NOVEL-V2.npy'


In [4]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

# TEMP LIMITER REMOVE WHEN READY
#file_names = [file_names[20]]

print(file_names)

['PROC-TEST2-DIAGONALS-L-LAR-160BPM-AUGMENT.xlsx', 'PROC-TEST2-DIAGONALS-L-LAR-40BPM-AUGMENT.xlsx', 'PROC-TEST2-DIAGONALS-L-LAR-80BPM.xlsx', 'PROC-TEST2-DIAGONALS-R-LAR-160BPM-AUGMENT.xlsx', 'PROC-TEST2-DIAGONALS-R-LAR-40BPM-AUGMENT.xlsx', 'PROC-TEST2-DIAGONALS-R-LAR-80BPM.xlsx', 'PROC-TEST2-SIDESTEPS-L-MED-140BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-L-MED-35BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-L-MED-70BPM.xlsx', 'PROC-TEST2-SIDESTEPS-R-MED-140BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-R-MED-35BPM-AUGMENT.xlsx', 'PROC-TEST2-SIDESTEPS-R-MED-70BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-90BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-100BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-MED-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-MED-200BPM-AUGMENT.xlsx', 'PRO

In [5]:
def normalizeSensorData(sensor_input):
    if(sensor_input < 0):
        return np.abs(sensor_input/360)
    else:
        return 0.5 + np.abs(sensor_input/360)


In [6]:
def df2Xy(df, windowSize=5, fetchAllSpeed=False):
  X, y = [], []


  for i in range( len(df) - windowSize ):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()
        
        # If j is the last element, fetch X_Vel and Z_Vel
        if (j == windowSize - 1) or (fetchAllSpeed == True):
            row_values.extend(df.loc[i + j, ['X_Vel', 'Z_Vel']].values.tolist())
        else:
            # Fill columns 5 and 6 with zeros
            row_values.extend([0.0, 0.0])

        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)


    # outputs: y labels
    label = df.loc[i + windowSize, ['X_Vel', 'Z_Vel']].values.tolist()

    y.append(label)


  return (np.array(X), np.array(y))

In [7]:
ALL_X_TEST_CURRVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X TESTing data
ALL_Y_TEST_CURRVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y training data

ALL_X_TEST_ALLVEL = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X TESTing data
ALL_Y_TEST_ALLVEL = np.empty((0, NUMBER_OF_CLASSES))  # List to store all Y TESTing data

In [10]:
def processData(ALL_X_TEST_CURRVEL, ALL_Y_TEST_CURRVEL, ALL_X_TEST_ALLVEL, ALL_Y_TEST_ALLVEL, NUMPY_DATA_FOLDER_FILE_PATH):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        # NORMALIZE THE SENSOR DATA
        df["L_Pitch"] = df["L_Pitch"].apply(normalizeSensorData)
        df["L_Roll"] = df["L_Roll"].apply(normalizeSensorData)
        df["R_Pitch"] = df["R_Pitch"].apply(normalizeSensorData)
        df["R_Roll"] = df["R_Roll"].apply(normalizeSensorData)

        xTest_CurrVel, yTest_CurrVel = df2Xy(df, WINDOW_SIZE, False)
        xTest_AllVel, yTest_AllVel = df2Xy(df, WINDOW_SIZE, True)

        ALL_X_TEST_CURRVEL = np.concatenate([ALL_X_TEST_CURRVEL, xTest_CurrVel])
        ALL_Y_TEST_CURRVEL = np.concatenate([ALL_Y_TEST_CURRVEL, yTest_CurrVel])

        ALL_X_TEST_ALLVEL = np.concatenate([ALL_X_TEST_ALLVEL, xTest_AllVel])
        ALL_Y_TEST_ALLVEL = np.concatenate([ALL_Y_TEST_ALLVEL, yTest_AllVel])

        
        print('xTest_CurrVel.shape:', xTest_CurrVel.shape, ' xTest_CurrVel.shape:', xTest_CurrVel.shape)
        print('ALL_X_TEST_CURRVEL.shape:', ALL_X_TEST_CURRVEL.shape, ' ALL_Y_TEST_CURRVEL.shape:', ALL_Y_TEST_CURRVEL.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TEST_CURRVEL_FILE_PATH, ALL_X_TEST_CURRVEL)
    np.save(ALL_Y_TEST_CURRVEL_FILE_PATH, ALL_Y_TEST_CURRVEL)
    np.save(ALL_X_TEST_ALLVEL_FILE_PATH, ALL_X_TEST_ALLVEL)
    np.save(ALL_Y_TEST_ALLVEL_FILE_PATH, ALL_Y_TEST_ALLVEL)


In [11]:
if not os.path.exists(ALL_X_TEST_CURRVEL_FILE_PATH):
    processData(ALL_X_TEST_CURRVEL, ALL_Y_TEST_CURRVEL, ALL_X_TEST_ALLVEL, ALL_Y_TEST_ALLVEL, NUMPY_DATA_FOLDER_FILE_PATH)

ALL_X_TEST_CURRVEL = np.load(ALL_X_TEST_CURRVEL_FILE_PATH)
ALL_Y_TEST_CURRVEL = np.load(ALL_Y_TEST_CURRVEL_FILE_PATH)
ALL_X_TEST_ALLVEL = np.load(ALL_X_TEST_ALLVEL_FILE_PATH)
ALL_Y_TEST_ALLVEL = np.load(ALL_Y_TEST_ALLVEL_FILE_PATH)

print(ALL_X_TEST_CURRVEL.shape)
print(ALL_Y_TEST_CURRVEL.shape)


xTest_CurrVel.shape: (3180, 10, 6)  xTest_CurrVel.shape: (3180, 10, 6)
ALL_X_TEST_CURRVEL.shape: (3180, 10, 6)  ALL_Y_TEST_CURRVEL.shape: (3180, 2)
-------------------------------
xTest_CurrVel.shape: (6365, 10, 6)  xTest_CurrVel.shape: (6365, 10, 6)
ALL_X_TEST_CURRVEL.shape: (9545, 10, 6)  ALL_Y_TEST_CURRVEL.shape: (9545, 2)
-------------------------------
xTest_CurrVel.shape: (3180, 10, 6)  xTest_CurrVel.shape: (3180, 10, 6)
ALL_X_TEST_CURRVEL.shape: (12725, 10, 6)  ALL_Y_TEST_CURRVEL.shape: (12725, 2)
-------------------------------
xTest_CurrVel.shape: (4013, 10, 6)  xTest_CurrVel.shape: (4013, 10, 6)
ALL_X_TEST_CURRVEL.shape: (16738, 10, 6)  ALL_Y_TEST_CURRVEL.shape: (16738, 2)
-------------------------------
xTest_CurrVel.shape: (8031, 10, 6)  xTest_CurrVel.shape: (8031, 10, 6)
ALL_X_TEST_CURRVEL.shape: (24769, 10, 6)  ALL_Y_TEST_CURRVEL.shape: (24769, 2)
-------------------------------
xTest_CurrVel.shape: (4013, 10, 6)  xTest_CurrVel.shape: (4013, 10, 6)
ALL_X_TEST_CURRVEL.shap

In [12]:
# assume the range is from -8 to 8
def normalizeXVelocity(velocity_input):
    return ((velocity_input + 8)/16)

# assume thhe range from 0 to 8
def normalizeZVelocity(velocity_input):
    return np.abs(velocity_input/8)

In [13]:
# given we have non normalize velocity, lets normalize velocity and hold it in our V1 data. 
ALL_X_TEST_CURRVEL_CURR_NORM1 = np.copy(ALL_X_TEST_CURRVEL)
ALL_Y_TEST_CURRVEL_CURR_NORM1 = np.copy(ALL_Y_TEST_CURRVEL)
ALL_X_TEST_CURRVEL_ALLVEL_NORM1 = np.copy(ALL_X_TEST_ALLVEL)
ALL_Y_TEST_CURRVEL_ALLVEL_NORM1 = np.copy(ALL_Y_TEST_ALLVEL)

#loop through each data training data
for i in range(ALL_X_TEST_CURRVEL_CURR_NORM1.shape[0]):
    #loop through each row in our WINDOW_SIZE
    for j in range(ALL_X_TEST_CURRVEL_CURR_NORM1.shape[1]):

        #normalize X annd Z velocity using our function
        #index 4 of X - X Velocity
        #index 5 of X - Z Velocity
        ALL_X_TEST_CURRVEL_CURR_NORM1[i,j,4] = normalizeXVelocity(ALL_X_TEST_CURRVEL_CURR_NORM1[i,j,4])
        ALL_X_TEST_CURRVEL_CURR_NORM1[i,j,5] = normalizeZVelocity(ALL_X_TEST_CURRVEL_CURR_NORM1[i,j,5])

        #index 0 of Y - X Velocity
        #index 1 of Y - Z Velocity
        ALL_Y_TEST_CURRVEL_CURR_NORM1[i,0] = normalizeXVelocity(ALL_Y_TEST_CURRVEL_CURR_NORM1[i,0])
        ALL_Y_TEST_CURRVEL_CURR_NORM1[i,1] = normalizeZVelocity( ALL_Y_TEST_CURRVEL_CURR_NORM1[i,1])

        ALL_X_TEST_CURRVEL_ALLVEL_NORM1[i,j,4] = normalizeXVelocity(ALL_X_TEST_CURRVEL_ALLVEL_NORM1[i,j,4])
        ALL_X_TEST_CURRVEL_ALLVEL_NORM1[i,j,5] = normalizeZVelocity(ALL_X_TEST_CURRVEL_ALLVEL_NORM1[i,j,5])

        ALL_Y_TEST_CURRVEL_ALLVEL_NORM1[i,0] = normalizeXVelocity(ALL_Y_TEST_CURRVEL_ALLVEL_NORM1[i,0])
        ALL_Y_TEST_CURRVEL_ALLVEL_NORM1[i,1] = normalizeZVelocity( ALL_Y_TEST_CURRVEL_ALLVEL_NORM1[i,1])


# Assuming your numpy array is called 'data_array'
np.save(ALL_X_TEST_CURRVEL_NORM1_FILE_PATH, ALL_X_TEST_CURRVEL_CURR_NORM1)
np.save(ALL_Y_TEST_CURRVEL_NORM1_FILE_PATH, ALL_Y_TEST_CURRVEL_CURR_NORM1)
np.save(ALL_X_TEST_ALLVEL_NORM1_FILE_PATH, ALL_X_TEST_CURRVEL_ALLVEL_NORM1)
np.save(ALL_Y_TEST_ALLVEL_NORM1_FILE_PATH, ALL_Y_TEST_CURRVEL_ALLVEL_NORM1)


In [3]:
ALL_X_TEST_CURRVEL_NORM = np.load(ALL_X_TEST_CURRVEL_NORM1_FILE_PATH)
ALL_Y_TEST_CURRVEL_NORM = np.load(ALL_Y_TEST_CURRVEL_NORM1_FILE_PATH)

ALL_X_TEST_NOVEL = ALL_X_TEST_CURRVEL_NORM[:, :, :4]

np.save(ALL_X_TEST_NOVEL_NORM_FILE_PATH, ALL_X_TEST_NOVEL)
np.save(ALL_Y_TEST_NOVEL_NORM_FILE_PATH, ALL_Y_TEST_CURRVEL_NORM)